In [1]:
import pandas as pd

gj_range = pd.read_csv('Geoje.csv')

In [2]:
keywords = ['여행', '분위기', '맛집', '감성', '체험', '가볼만한 곳']
categorys = ['CT1', 'AT4', 'CE7']
categorys_info = {'CT1' : '문화시설', 'AT4' : '관광명소', 'CE7' : '카페'}

In [3]:
gj = gj_range['이름']

In [4]:
import json
import requests

def search_result(keyword, category, geoje_name):
    result = []
    # REST 키
    headers = {"Authorization": "카카오키"}
    # 파라미터
    params = {"query" : f"거제시 {geoje_name}{keyword}", "page" : 1, "category_group_code" : f"{category}"}
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"

    while True:
        # GET을 이용하여 획득
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            # Json을 이용하여 해제
            doc = json.loads(res.text)
            result.extend(doc['documents'])
            if doc['meta']['is_end'] == True:
                break
            else:
                params['page'] += 1
    return result

# GET을 이용하여 획득
#res = requests.get(url, headers=headers, params=params)
# Json을 이용하여 해제
#doc = json.loads(res.text)
# 200일 경우 정상
#res.status_code

In [5]:
import pandas as pd
from tqdm.notebook import tqdm

def search_df():
    results = []
    for idx in tqdm(gj):
        for category in categorys:
            for key in keywords:
                r = pd.DataFrame(search_result(key, category, idx))
                r['keyword'] = key
                results.append(r.copy())
    return pd.concat(results).reset_index(drop=True)

In [6]:
geoje_poi = search_df()
geoje_poi.to_csv('geoje_poi.csv',index=False)

  0%|          | 0/18 [00:00<?, ?it/s]

In [7]:
geoje_poi.head(2)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,여행,경남 거제시 일운면 구조라리 423-1,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",,11026203,055-639-3000,구조라해수욕장,http://place.map.kakao.com/11026203,경남 거제시 일운면 거제대로 2056,128.692890014526,34.8069653420412
1,여행,경남 거제시 일운면 와현리 산 109,AT4,관광명소,"여행 > 관광,명소 > 수목원,식물원",,8021411,055-681-4541,외도보타니아,http://place.map.kakao.com/8021411,,128.71165107261427,34.76929638286456


In [8]:
import pandas as pd

api_poi = pd.read_csv('geoje_poi.csv', index_col=False)
api_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2459 entries, 0 to 2458
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              2459 non-null   object 
 1   address_name         2459 non-null   object 
 2   category_group_code  2459 non-null   object 
 3   category_group_name  2459 non-null   object 
 4   category_name        2459 non-null   object 
 5   distance             0 non-null      float64
 6   id                   2459 non-null   int64  
 7   phone                607 non-null    object 
 8   place_name           2459 non-null   object 
 9   place_url            2459 non-null   object 
 10  road_address_name    984 non-null    object 
 11  x                    2459 non-null   float64
 12  y                    2459 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 249.9+ KB


In [9]:
pd.DataFrame(api_poi['keyword'].value_counts())

,keyword
여행,810
가볼만한 곳,810
맛집,809
감성,26
체험,4


In [10]:
pd.DataFrame(api_poi['category_group_name'].value_counts())

,category_group_name
관광명소,1621
카페,837
문화시설,1


In [11]:
included = {}
del_index = []
for idx, row in api_poi.iterrows():
    id = row['id']
    temp = api_poi[api_poi['id'] == id].copy()
    cnt = len(temp)
    if id not in included and cnt > 1:
        included[id] = True
        for i, r in temp.iterrows():
            if idx == i:
                continue
            else:
                api_poi.loc[idx, 'keyword'] = api_poi.loc[idx, 'keyword'] + ',' + r['keyword']
                del_index.append(i)

In [12]:
# 중복 id의 수, 삭제할 인덱스의 수
len(included.keys()), len(del_index)

(653, 1360)

In [13]:
# 중복 id의 인덱스 삭제
api_poi_del = api_poi.drop(del_index, axis=0)

In [14]:
api_poi_del.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1099 entries, 0 to 2453
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              1099 non-null   object 
 1   address_name         1099 non-null   object 
 2   category_group_code  1099 non-null   object 
 3   category_group_name  1099 non-null   object 
 4   category_name        1099 non-null   object 
 5   distance             0 non-null      float64
 6   id                   1099 non-null   int64  
 7   phone                383 non-null    object 
 8   place_name           1099 non-null   object 
 9   place_url            1099 non-null   object 
 10  road_address_name    659 non-null    object 
 11  x                    1099 non-null   float64
 12  y                    1099 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 120.2+ KB


In [15]:
api_poi_del.to_csv('geoje_poi1.csv', index=False)

In [16]:
# id 중복 제거 데이터 확인
import pandas as pd

api_poi_del = pd.read_csv('geoje_poi1.csv', index_col=False)
api_poi_del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1099 entries, 0 to 1098
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              1099 non-null   object 
 1   address_name         1099 non-null   object 
 2   category_group_code  1099 non-null   object 
 3   category_group_name  1099 non-null   object 
 4   category_name        1099 non-null   object 
 5   distance             0 non-null      float64
 6   id                   1099 non-null   int64  
 7   phone                383 non-null    object 
 8   place_name           1099 non-null   object 
 9   place_url            1099 non-null   object 
 10  road_address_name    659 non-null    object 
 11  x                    1099 non-null   float64
 12  y                    1099 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 111.7+ KB


In [17]:
len(api_poi_del['id'].unique())

1099

In [18]:
pd.DataFrame(api_poi_del['category_group_name'].value_counts())

,category_group_name
카페,626
관광명소,472
문화시설,1


In [19]:
api_poi_del

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,"여행,가볼만한 곳,여행,가볼만한 곳,여행,가볼만한 곳",경남 거제시 일운면 구조라리 423-1,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",NaN,11026203,055-639-3000,구조라해수욕장,http://place.map.kakao.com/11026203,경남 거제시 일운면 거제대로 2056,128.692890,34.806965
1,"여행,가볼만한 곳",경남 거제시 일운면 와현리 산 109,AT4,관광명소,"여행 > 관광,명소 > 수목원,식물원",NaN,8021411,055-681-4541,외도보타니아,http://place.map.kakao.com/8021411,NaN,128.711651,34.769296
2,"여행,가볼만한 곳,여행,가볼만한 곳",경남 거제시 일운면 망치리,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",NaN,7862496,NaN,망치몽돌해수욕장,http://place.map.kakao.com/7862496,NaN,128.667764,34.812460
3,"여행,가볼만한 곳,여행,가볼만한 곳,여행,가볼만한 곳",경남 거제시 일운면 와현리 431-2,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",NaN,11297994,055-639-3000,와현모래숲해수욕장,http://place.map.kakao.com/11297994,경남 거제시 일운면 와현해변길 15,128.708607,34.812672
4,"여행,가볼만한 곳,여행,가볼만한 곳,여행,가볼만한 곳",경남 거제시 일운면 옥림리 57-1,AT4,관광명소,"여행 > 관광,명소 > 섬",NaN,8559153,NaN,지심도,http://place.map.kakao.com/8559153,경남 거제시 일운면 지심도길 31-2,128.749383,34.817140
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,맛집,경남 거제시 양정동 208-4,CE7,카페,음식점 > 카페,NaN,693746293,NaN,꽃다비카페,http://place.map.kakao.com/693746293,경남 거제시 양정1길 45,128.646210,34.869152
1095,맛집,경남 거제시 양정동 330-1,CE7,카페,음식점 > 카페,NaN,19046427,NaN,커피향기,http://place.map.kakao.com/19046427,경남 거제시 양정2길 81,128.645399,34.872120
1096,맛집,경남 거제시 양정동 2,CE7,카페,음식점 > 카페,NaN,19046638,NaN,J커피점,http://place.map.kakao.com/19046638,경남 거제시 양정3길 74,128.659529,34.869984
1097,맛집,경남 거제시 양정동 936-100,CE7,카페,음식점 > 카페,NaN,256152035,NaN,아인츠디자인카페,http://place.map.kakao.com/256152035,경남 거제시 제산로 28-2,128.638484,34.890029
